This code has not been fully tested. Results may vary.

In [ ]:
from sim_EMR_v2 import build_trial_set
idCount = 120
per_arm = 10
num_chunks = 6
for chunks in range(num_chunks):
    build_trial_set(patients_per_arm = per_arm,placebo_efficacy = 0, drug_efficacy = 0.39, baseline_dur = 2, test_dur = 3,
                    fname_raw = 'RCT2_raw',fname_sum = 'RCT2_sum',true_sum='RCT2_true',IDstart=idCount,noSum=False,rawTF=True,
                    LLM_gen_name='llama3:8b')
    idCount += per_arm*2

In [1]:
from sim_EMR_v2 import build_trial_set, make_one_patient
name = '123'
idnum=1
drug_name='abc1'
baseline_sz=2
test_sz=4
baseline_dur=2
test_dur=3
selected_sx_str='sad,tired,cranky'
noSum=True
LLM_gen_name='llama3:8b'
LLM_sum_name='mistral'
arm=1
R1,R2 = make_one_patient(name = name, idnum= idnum, drug_name=drug_name , baseline_sz=baseline_sz, 
                                test_sz=test_sz, baseline_dur=baseline_dur, 
                                test_dur=test_dur,selected_sx_str = selected_sx_str,noSum=noSum,
                                LLM_gen_name=LLM_gen_name,LLM_sum_name=LLM_sum_name,arm=arm)

Time out!

SystemExit: 

/Users/danielgoldenholz/miniforge3/envs/deept2/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
# used for SourceSpikeNet
from statsmodels.stats.power import FTestAnovaPower

# Define the parameters for the power analysis
effect_size = 0.20  # medium effect size
alpha = 0.05  # significance level
power = 0.8  # desired power
num_groups = 20  # number of groups (e.g., 20 master IED generators)

# Create an instance of the FTestAnovaPower class
anova_power = FTestAnovaPower()

# Perform the power analysis to calculate the required sample size per group
required_sample_size_per_group = anova_power.solve_power(effect_size=effect_size, 
                                                         alpha=alpha, 
                                                         power=power, 
                                                         k_groups=num_groups)

# Calculate the total sample size required
total_sample_size = required_sample_size_per_group * num_groups

# Print the results
print(f"Required sample size per group: {required_sample_size_per_group:.2f}")
print(f"Total sample size required: {total_sample_size:.2f}")

Required sample size per group: 529.11
Total sample size required: 10582.30


In [10]:
# used for SourceSpikeNet
from statsmodels.stats.power import TTestIndPower

# Define the parameters for the power analysis
effect_size = (60-16) / 40  # calculated Cohen's d
print(effect_size)
alpha = 0.05  # significance level
power = 0.8  # desired power
num_groups = 2  # comparing two conditions: SpikeNet and SourceSpikeNet

# Create an instance of the TTestIndPower class
power_analysis = TTestIndPower()

# Perform the power analysis to calculate the required sample size per group
required_sample_size_per_group = power_analysis.solve_power(effect_size=effect_size, 
                                                            alpha=alpha, 
                                                            power=power)

# Calculate the total sample size required
total_sample_size = required_sample_size_per_group * num_groups

# Print the results
print(f"Required sample size per group: {required_sample_size_per_group:.2f}")
print(f"Total sample size required: {total_sample_size:.2f}")

1.1
Required sample size per group: 14.00
Total sample size required: 28.00


In [2]:
import pandas as pd

# Create a DataFrame with the data
f1 = '/Users/danielgoldenholz/icloud/Latest/LLM-derive-realEMR/daniel_g_project/meds_all.csv'
f2 = '/Users/danielgoldenholz/icloud/Latest/LLM-derive-realEMR/MGB_Deidentified_Notes_March12th2024/mgb_notes_2015_metadata.csv'
f1_hasnotes = '/Users/danielgoldenholz/icloud/Latest/LLM-derive-realEMR/daniel_g_project/meds_all_hasnotes.csv'

# first make a reduced version with only has_notes==TRUE
df1 = pd.read_csv(f1)

df1_hasnotes = df1[df1['has_notes']==True]
df1_hasnotes.to_csv(f1_hasnotes,index=False)



/var/folders/9f/mx3t8lrj4719csmgr9pw7n1r0000gr/T/ipykernel_65452/1616086225.py:7: DtypeWarning: Columns (4,5,6,8,10,13,14,15,18,19,20,21,22,23,24,28,29,30,31,32,33,34,35,36,38,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(f1)


In [8]:
from langchain_community.llms import Ollama
import time
import gc   
from sim_EMR_v2 import getLLM_response_withLLM

def tryMe():
    gc.collect() # clean up the memory
    time_out = 120  # max seconds for generating one response. This is a safety feature to prevent too much output.
    temp_sum = 1
    #LLM_sum_name = 'llama3:8b'
    LLM_sum_name = 'phi3:medium-128k'
    # Create a client object
    LLM_sum = Ollama(model=LLM_sum_name,temperature=temp_sum)
    prompt = 'I have a 38 year old female with drug resistant epilepsy, hypothyroidism, and migraines. Explain what is the most likely to kill this patient and why.'
    getLLM_response_withLLM(thePrompt=prompt,llm=LLM_sum,time_out=time_out)

tryMe()

...

In [45]:
import subprocess
import tqdm.auto as tqdm
from datetime import datetime
#df2 = pd.read_csv(f2)

# Define your array of filenames and tar archive name
IDnum = 1
print(df1_hasnotes.BDSPPatientID.iloc[IDnum])
#print(df1_hasnotes.StartDTS.iloc[IDnum])
ptID = df1_hasnotes.BDSPPatientID.iloc[IDnum]
startDTS = df1_hasnotes.StartDTS.iloc[IDnum]
endDTS = df1_hasnotes.EndDTS.iloc[IDnum]
#startDTS = startDTS.rstrip('0')
#endDTS = endDTS.rstrip('0')

date_format = "%Y-%m-%d %H:%M:%S.%f"
startDTS = startDTS.rstrip('0')
endDTS = endDTS.rstrip('0')

# If the last character is a dot, add one zero
if startDTS[-1] == '.':
    startDTS += '0'
if endDTS[-1] == '.':
    endDTS += '0'
startDT = datetime.strptime(startDTS, date_format)
endDT = datetime.strptime(endDTS,date_format)

thisPt = df2[df2['BDSPPatientID']==ptID]
tarfile = '/Users/danielgoldenholz/icloud/Latest/LLM-derive-realEMR/MGB_Deidentified_Notes_March12th2024/mgb_notes_2015.tar' 
output_dir='/Users/danielgoldenholz/icloud/Latest/LLM-derive-realEMR/MGB_Deidentified_Notes_March12th2024/'
date_format2 = "%Y%m%d"


# Loop through each filename and extract it from the tar file
for index, fnameLine in tqdm.tqdm(thisPt.iterrows(), total=thisPt.shape[0]):
    fname = fnameLine.DeidentifiedName
    date_text = str(fnameLine.ContactDate)

    # Convert the text to a datetime object
    date_object = datetime.strptime(date_text, date_format2)

    #print( str(startDT) + ' ' + str(date_object) + ' ' + str(endDT))
    if date_object >= startDT and date_object <= endDT:
        command = ['tar','-xvf',tarfile,'-C',output_dir,f'2015/{fname}']
        subprocess.run(command)
        

120911044


0it [00:00, ?it/s]

Notes_13242081857_1031820141_20150706.txt


x 2015/Notes_13242081857_1031820141_20150706.txt


Notes_13242081857_1031864355_20150706.txt


...

x 2015/Notes_13226473846_1036483866_20150708.txt


Notes_13226473846_1036976251_20150709.txt


x 2015/Notes_13226473846_1036976251_20150709.txt


Notes_13243564447_1040364020_20150712.txt


x 2015/Notes_13243564447_1040364020_20150712.txt
